In [7]:
using Random, LinearAlgebra, Plots
# bring packages into main namespace
using DataFrames             # Data tables are called "DataFrames"
using StatsPlots             # load plotting packages 
using Statistics             # basic statistical functions
using CSV                    # tools for working with CSV files
using Plots, Random, LinearAlgebra, Statistics, SparseArrays
include("proxgrad.jl")
pyplot()


Plots.PyPlotBackend()

In [40]:
dataset = CSV.read("Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv") #, limit=10000) # read 10000 rows from the csv
dataset[:when] = dataset[:Year] .+ ((dataset[:quarter] .-1) ./ 4)
n = size(dataset,1)
@show n
head(dataset)

n = 213175


,tbl,Year,quarter,citymarketid_1,citymarketid_2,city1
,String,Int64,Int64,Int64,Int64,String
1,Table 1a,2010,1,34614,33195,"Salt Lake City, UT"
2,Table 1a,1998,4,30189,31703,"Colorado Springs, CO"
3,Table 1a,1998,4,30198,30852,"Pittsburgh, PA"
4,Table 1a,2009,3,32211,32575,"Las Vegas, NV"
5,Table 1a,1993,4,30255,30852,"Huntsville, AL"
6,Table 1a,2010,4,33198,32575,"Kansas City, MO"


In [64]:
# One featurization
function get_Xy(dataset)
    X = [dataset[:when] dataset[:airportid_1] dataset[:airportid_2] ones(size(dataset,1))]
    y = dataset[:fare]
    return X,y
end
X, y = get_Xy(dataset)

w = X \ y

function crossVal(featurizer, opt; k=10)
    X,y = featurizer(dataset)
    n = 75
    r = Int(round(n / k))
    idxs = rand(1:n,n) # to shuffle the dataset
    for i = 1:k
        val_idxs = r*(i-1)+1:r*i
        tr_low = 1:r*(i-1)
        tr_high = r*i+1:n
        if (i == 1)
            tr_idxs = tr_high
        elseif (i == k)
            tr_idxs = tr_low
        else
            tr_idxs = [tr_low ; tr_high ]
        end
        @show i, val_idxs
        @show tr_idxs
        X_tr = X[idxs[tr_idxs],:]
        y_tr = y[idxs[tr_idxs],:]
        
        X_val = X[idxs[val_idxs],:]
        y_val = y[idxs[val_idxs],:]
    end
end
crossVal(0,0)

(i, val_idxs) = (1, 1:8)
tr_idxs = 9:75
(i, val_idxs) = (2, 9:16)
tr_idxs = [1, 2, 3, 4, 5, 6, 7, 8, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
(i, val_idxs) = (3, 17:24)
tr_idxs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
(i, val_idxs) = (4, 25:32)
tr_idxs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75]
(i, val_idxs) = (5, 33:40)
tr_idxs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 